## The purpose of this notebook is to replicate the results of the oakland-only test with all zones in Pittsburgh.
   - I reccomend that when you run this, you make sure to have your display turned on.  Sometimes, when display shuts off, jupter notebook loses the port and stops sending the output (particularly during the block where we iterate through all the transactions).  
        - It will continue to run, but you might stop seeing output.  
        - An easy sanity check would be to watch your file explorer.  No indiviudal file should take more than ~45min or so
        - Alternatively, watch the activity monitor to make sure python is hogging basically your entire CPU
 
   - To see the next steps after this notebook, scroll to bottom of file

In [26]:
import pandas as pd
import math
import numpy as np
import datetime
import time

### This first section of the notebook creates the dataframe of buckets for each zone.
   - Previously, this was captured using http request to the aggregates database from the data authority. 
   - Instead, we generate time series over the course of a year in ten minute intervals.  
   
   - This solves the problem of having to query over the 50000 limit from the database.

In [27]:
#create dictionary of all zones and their space count
zone_df = pd.read_csv("../data/zones.csv")
zone_dict = {}

for index, row in zone_df.iterrows():    
    if " - " not in row['zone']:
        continue
        
    if row['zone'] in zone_dict:
        zone_dict[row['zone']] = max(zone_dict[row['zone']], row['spaces'])
    else:
        zone_dict[row['zone']] = row['spaces']
        
zone_dict

{'301 - Sheridan Harvard Lot': 41,
 '302 - Sheridan Kirkwood Lot': 114,
 '304 - Tamello Beatty Lot': 76,
 '307 - Eva Beatty Lot': 130,
 '311 - Ansley Beatty Lot': 23,
 '314 - Penn Circle NW Lot': 125,
 '321 - Beacon Bartlett Lot': 69,
 '322 - Forbes Shady Lot': 59,
 '323 - Douglas Phillips Lot': 45,
 '324 - Forbes Murray Lot': 72,
 '325 - JCC/Forbes Lot': 69,
 '328 - Ivy Bellefonte Lot': 74,
 '331 - Homewood Zenith Lot': 24,
 '334 - Taylor Street Lot': 26,
 '335 - Friendship Cedarville Lot': 80,
 '337 - 52nd & Butler Lot': 12,
 '338 - 42nd & Butler Lot': 22,
 '341 - 18th & Sidney Lot': 45,
 '342 - East Carson Lot': 35,
 '343 - 19th & Carson Lot': 27,
 '344 - 18th & Carson Lot': 41,
 '345 - 20th & Sidney Lot': 80,
 '351 - Brownsville & Sandkey Lot': 80,
 '354 - Walter/Warrington Lot': 15,
 '355 - Asteroid Warrington Lot': 13,
 '357 - Shiloh Street Lot': 73,
 '361 - Brookline Lot': 47,
 '363 - Beechview Lot': 17,
 '369 - Main/Alexander Lot': 29,
 '371 - East Ohio Street Lot': 88,
 '375 -

In [28]:
#create a base dataframe for a zone containing: start | day_of_week | date | time

start = '2018-01-01 00:00'
end = '2018-12-31 23:50'

#tz is VERY IMPORTANT or else daylight savings will ruin your day
buckets = pd.date_range(start=start, end=end, freq='10min', tz='US/Eastern')
buckets = buckets.to_frame(index=False, name='start')

buckets['start'] = pd.to_datetime(buckets['start'])
buckets['day_of_week'] = buckets['start'].dt.day_name()
buckets['date'] = buckets['start'].dt.strftime('%m-%d')
buckets['time'] = buckets['start'].dt.time

buckets

,start,day_of_week,date,time
0,2018-01-01 00:00:00-05:00,Monday,01-01,00:00:00
1,2018-01-01 00:10:00-05:00,Monday,01-01,00:10:00
2,2018-01-01 00:20:00-05:00,Monday,01-01,00:20:00
3,2018-01-01 00:30:00-05:00,Monday,01-01,00:30:00
4,2018-01-01 00:40:00-05:00,Monday,01-01,00:40:00
...,...,...,...,...
52555,2018-12-31 23:10:00-05:00,Monday,12-31,23:10:00
52556,2018-12-31 23:20:00-05:00,Monday,12-31,23:20:00
52557,2018-12-31 23:30:00-05:00,Monday,12-31,23:30:00
52558,2018-12-31 23:40:00-05:00,Monday,12-31,23:40:00


In [29]:
#convert the starter dataframe into a unique dataframe for each zone
zone_buckets = {}
for zone in zone_dict:
        df = pd.DataFrame(buckets)
        df['total_spaces'] = zone_dict[zone]
        df['free_spaces'] = zone_dict[zone]
        df['zone'] = zone
        
        cols = df.columns.tolist()
        cols = cols[-1:] + cols[:-1]
        zone_buckets[zone] = df[cols] #I like having the zone on the left 
        
#printing an arbitrary zone just to see what we're expecting the dataframe to look like for each zone
zone_buckets['407 - Oakland 1']

#as an interesting note, each dataframe is approximately 14 MB.  Uncomment these lines to see
    # from sys import getsizeof
    # getsizeof(zone_buckets['407 - Oakland 1'])

,zone,start,day_of_week,date,time,total_spaces,free_spaces
0,407 - Oakland 1,2018-01-01 00:00:00-05:00,Monday,01-01,00:00:00,227,227
1,407 - Oakland 1,2018-01-01 00:10:00-05:00,Monday,01-01,00:10:00,227,227
2,407 - Oakland 1,2018-01-01 00:20:00-05:00,Monday,01-01,00:20:00,227,227
3,407 - Oakland 1,2018-01-01 00:30:00-05:00,Monday,01-01,00:30:00,227,227
4,407 - Oakland 1,2018-01-01 00:40:00-05:00,Monday,01-01,00:40:00,227,227
...,...,...,...,...,...,...,...
52555,407 - Oakland 1,2018-12-31 23:10:00-05:00,Monday,12-31,23:10:00,227,227
52556,407 - Oakland 1,2018-12-31 23:20:00-05:00,Monday,12-31,23:20:00,227,227
52557,407 - Oakland 1,2018-12-31 23:30:00-05:00,Monday,12-31,23:30:00,227,227
52558,407 - Oakland 1,2018-12-31 23:40:00-05:00,Monday,12-31,23:40:00,227,227


### Preparing to go through individual transactions.  The trick here is:
   - Go zone by zone to elminate unneccesary comparisons
   - Subtract in Slices instead of individually
   - "Roll" over the buckets so we don't do unncessary checks
    
##### Lets start by verifying that daylight savings cooperates with our bucket methodology
   - to understand this problem, look at it in  test.csv file around date 3-11 and 11-04.
   - we'll check fall daylight savings and spring daylight savings to make sure that time deltas between consecutive buckets go up by ten minutes.  
       - consider someone who pays for 30 minutes of parking 10 minutes before 'spring forward'.  We want to make sure that by subtracting a start/end time, we can still figure out the right number of buckets to move along.

In [30]:
fall_daylight_savings = zone_buckets['407 - Oakland 1'].iloc[44210:44220]
fall_daylight_savings

,zone,start,day_of_week,date,time,total_spaces,free_spaces
44210,407 - Oakland 1,2018-11-04 01:20:00-04:00,Sunday,11-04,01:20:00,227,227
44211,407 - Oakland 1,2018-11-04 01:30:00-04:00,Sunday,11-04,01:30:00,227,227
44212,407 - Oakland 1,2018-11-04 01:40:00-04:00,Sunday,11-04,01:40:00,227,227
44213,407 - Oakland 1,2018-11-04 01:50:00-04:00,Sunday,11-04,01:50:00,227,227
44214,407 - Oakland 1,2018-11-04 01:00:00-05:00,Sunday,11-04,01:00:00,227,227
44215,407 - Oakland 1,2018-11-04 01:10:00-05:00,Sunday,11-04,01:10:00,227,227
44216,407 - Oakland 1,2018-11-04 01:20:00-05:00,Sunday,11-04,01:20:00,227,227
44217,407 - Oakland 1,2018-11-04 01:30:00-05:00,Sunday,11-04,01:30:00,227,227
44218,407 - Oakland 1,2018-11-04 01:40:00-05:00,Sunday,11-04,01:40:00,227,227
44219,407 - Oakland 1,2018-11-04 01:50:00-05:00,Sunday,11-04,01:50:00,227,227


In [31]:
t1 = fall_daylight_savings.iloc[4]['start']
t2 = fall_daylight_savings.iloc[3]['start']

#confirm that even after "Fall Back", subtracting consecutive times shows a positive gain by one bucket
print(t1, " - ", t2, " = ", (t1-t2))


2018-11-04 01:00:00-05:00  -  2018-11-04 01:50:00-04:00  =  0 days 00:10:00


In [32]:
spring_daylight_savings = zone_buckets['407 - Oakland 1'].iloc[9945:9955]
spring_daylight_savings

,zone,start,day_of_week,date,time,total_spaces,free_spaces
9945,407 - Oakland 1,2018-03-11 01:30:00-05:00,Sunday,03-11,01:30:00,227,227
9946,407 - Oakland 1,2018-03-11 01:40:00-05:00,Sunday,03-11,01:40:00,227,227
9947,407 - Oakland 1,2018-03-11 01:50:00-05:00,Sunday,03-11,01:50:00,227,227
9948,407 - Oakland 1,2018-03-11 03:00:00-04:00,Sunday,03-11,03:00:00,227,227
9949,407 - Oakland 1,2018-03-11 03:10:00-04:00,Sunday,03-11,03:10:00,227,227
9950,407 - Oakland 1,2018-03-11 03:20:00-04:00,Sunday,03-11,03:20:00,227,227
9951,407 - Oakland 1,2018-03-11 03:30:00-04:00,Sunday,03-11,03:30:00,227,227
9952,407 - Oakland 1,2018-03-11 03:40:00-04:00,Sunday,03-11,03:40:00,227,227
9953,407 - Oakland 1,2018-03-11 03:50:00-04:00,Sunday,03-11,03:50:00,227,227
9954,407 - Oakland 1,2018-03-11 04:00:00-04:00,Sunday,03-11,04:00:00,227,227


In [33]:
t1 = spring_daylight_savings.iloc[3]['start']
t2 = spring_daylight_savings.iloc[2]['start']

#confirm that even after "Spring Forward", subtracting consecutive times shows a positive gain of one bucket
print(t1, " - ", t2, " = ", (t1-t2))

2018-03-11 03:00:00-04:00  -  2018-03-11 01:50:00-05:00  =  0 days 00:10:00


### Now we can go through all the transactions

In [34]:
bucket_delta = pd.Timedelta(value=10, unit="min")
transactions_2018 = pd.read_csv("../../unholy data dump/2018--transactions.csv")

#extract what we want from the columns and set them as datetime objects
transactions_2018 = transactions_2018[['zone', 'payment_start_utc', 'payment_end_utc', 'amount']]
transactions_2018['payment_start_utc'] = pd.to_datetime(transactions_2018['payment_start_utc']).dt.tz_localize('UTC')
transactions_2018['payment_end_utc'] = pd.to_datetime(transactions_2018['payment_end_utc']).dt.tz_localize('UTC')

#convert to eastern time
transactions_2018['payment_start_east'] = transactions_2018['payment_start_utc'].dt.tz_convert('US/Eastern')
transactions_2018['payment_end_east'] = transactions_2018['payment_end_utc'].dt.tz_convert('US/Eastern')

#determine how many buckets this car is here for
transactions_2018['start_bucket'] = transactions_2018['payment_start_east'].dt.floor('10min')
transactions_2018['num_buckets'] = (transactions_2018['payment_end_east'] - transactions_2018['start_bucket'])/bucket_delta
transactions_2018['num_buckets'] = transactions_2018['num_buckets'].apply(np.ceil).astype(int)

transactions_2018

,zone,payment_start_utc,payment_end_utc,amount,payment_start_east,payment_end_east,start_bucket,num_buckets
0,421 - NorthSide,2018-01-01 05:24:00+00:00,2018-01-01 16:59:00+00:00,4.00,2018-01-01 00:24:00-05:00,2018-01-01 11:59:00-05:00,2018-01-01 00:20:00-05:00,70
1,412 - East Liberty,2018-01-01 06:10:00+00:00,2018-01-01 14:59:00+00:00,3.00,2018-01-01 01:10:00-05:00,2018-01-01 09:59:00-05:00,2018-01-01 01:10:00-05:00,53
2,421 - NorthSide,2018-01-01 06:28:00+00:00,2018-01-01 16:59:00+00:00,4.00,2018-01-01 01:28:00-05:00,2018-01-01 11:59:00-05:00,2018-01-01 01:20:00-05:00,64
3,403 - Uptown,2018-01-01 06:18:00+00:00,2018-01-01 14:59:00+00:00,3.00,2018-01-01 01:18:00-05:00,2018-01-01 09:59:00-05:00,2018-01-01 01:10:00-05:00,53
4,335 - Friendship Cedarville Lot,2018-01-01 11:54:00+00:00,2018-01-01 21:54:00+00:00,9.00,2018-01-01 06:54:00-05:00,2018-01-01 16:54:00-05:00,2018-01-01 06:50:00-05:00,61
...,...,...,...,...,...,...,...,...
7740289,341 - 18th & Sidney Lot,2019-01-01 02:48:00+00:00,2019-01-01 03:00:00+00:00,1.00,2018-12-31 21:48:00-05:00,2018-12-31 22:00:00-05:00,2018-12-31 21:40:00-05:00,2
7740290,345 - 20th & Sidney Lot,2019-01-01 02:33:00+00:00,2019-01-01 03:00:00+00:00,1.00,2018-12-31 21:33:00-05:00,2018-12-31 22:00:00-05:00,2018-12-31 21:30:00-05:00,3
7740291,341 - 18th & Sidney Lot,2019-01-01 02:26:00+00:00,2019-01-01 03:00:00+00:00,1.00,2018-12-31 21:26:00-05:00,2018-12-31 22:00:00-05:00,2018-12-31 21:20:00-05:00,4
7740292,344 - 18th & Carson Lot,2019-01-01 02:36:00+00:00,2019-01-01 03:00:00+00:00,1.00,2018-12-31 21:36:00-05:00,2018-12-31 22:00:00-05:00,2018-12-31 21:30:00-05:00,3


In [18]:
# #test for iloc to see which index has the bucket start time
# b = zone_buckets['407 - Oakland 1']
# b.iloc[:,1]

# #test for timing the loop
# t1 = time.time()
# for i in range(0, 1000000):
#     pass
# t2 = time.time()
# minutes = round((t2-t1) / 60,5)
# print("Elapsed: ", minutes)

# #test saving to a file
# zone = '301 - Sheridan Harvard Lot'
# file_name = '../data/occupancies/' + zone + '.csv'
# zone_buckets[zone].to_csv(file_name)


In [119]:
transactions_by_zone = transactions_2018.groupby('zone')
total_elapsed = 0

for zone in zone_dict:        
    transactions = transactions_by_zone.get_group(zone)
    buckets = zone_buckets[zone]
    t1 = time.time()
    
    start_index = 0
    for index, transaction in transactions.iterrows():
        # uncomment this line for 7 million + lines of output, or trust that it works
        #print("Proccesing transaction: ", index, " | STARTS: ", transaction['payment_start_east'], " | BUCKETS: ", transaction['num_buckets'])
        start_index = start_index + int((transaction['start_bucket'] - buckets.iloc[start_index,1])/bucket_delta)
        end_index = start_index + transaction['num_buckets']
        buckets.iloc[start_index:end_index, -1] -= 1
    
    zone_buckets[zone] = buckets
    
    t2 = time.time()
    elapsed = round((t2-t1)/60,2)
    total_elapsed += elapsed
    
    file_name = '../data/occupancies/' + zone.replace('/', '') + '.csv'
    print("*******************************")
    print("Time elapsed: ", elapsed, " minutes")
    print("Creating CSV: ", file_name)
    print("*******************************")
    zone_buckets[zone].to_csv(file_name)
    
print("Bucket generation completed!")
print("Total time: ", total_elapsed, "minutes")

### Now that we have generated all of the 'free spaces', we are going to generate occupancy as a ratio and smush all the zones together into one .csv file 

In [42]:
#load in all the individual csv's in case something bad happened during processing
for zone in zone_dict:
    file_name = '../data/occupancies/' + zone.replace('/', '') + '.csv'
    zone_buckets[zone] = pd.read_csv(file_name)

#combine all the individual zones into one big csv
all_zone_buckets = pd.concat(zone_buckets.values(), ignore_index = True, keys=None)
all_zone_buckets = all_zone_buckets.sort_values(by='start')

#calculate occupancy to three decimal places and make sure its nonnegative
all_zone_buckets['occupancy'] = all_zone_buckets['free_spaces'] / all_zone_buckets['total_spaces']
all_zone_buckets['occupancy'] = all_zone_buckets['occupancy'].round(3)
temp = all_zone_buckets['occupancy']
temp[temp < 0] = 0
all_zone_buckets['occupancy'] = temp

#gather the desired columns
all_zone_buckets = all_zone_buckets[['zone', 'start', 'day_of_week', 'date', 'time', 'total_spaces', 'free_spaces', 'occupancy']]
all_zone_buckets

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,zone,start,day_of_week,date,time,total_spaces,free_spaces,occupancy
0,301 - Sheridan Harvard Lot,2018-01-01 00:00:00-05:00,Monday,01-01,00:00:00,41,41,1.0
2312640,414 - Mellon Park,2018-01-01 00:00:00-05:00,Monday,01-01,00:00:00,76,76,1.0
2733120,422 - Northshore,2018-01-01 00:00:00-05:00,Monday,01-01,00:00:00,237,237,1.0
2365200,415 - SS & SSW,2018-01-01 00:00:00-05:00,Monday,01-01,00:00:00,707,707,1.0
946080,342 - East Carson Lot,2018-01-01 00:00:00-05:00,Monday,01-01,00:00:00,35,35,1.0
...,...,...,...,...,...,...,...,...
2365199,414 - Mellon Park,2018-12-31 23:50:00-05:00,Monday,12-31,23:50:00,76,76,1.0
578159,325 - JCC/Forbes Lot,2018-12-31 23:50:00-05:00,Monday,12-31,23:50:00,69,69,1.0
2417759,415 - SS & SSW,2018-12-31 23:50:00-05:00,Monday,12-31,23:50:00,707,707,1.0
2470319,416 - Carrick,2018-12-31 23:50:00-05:00,Monday,12-31,23:50:00,65,65,1.0


In [43]:
#save to csv file
all_zone_buckets.to_csv('../data/occupancies/ALL_zone_occupancies.csv')

## Next steps
 - Do nearly the same thing except at the street level
     - probably much more computationally expensive
     - will require setting up the mapping from meter to street (not that hard)
     - also need to estimate the capacity of a street by taking the max number of cars over all buckets in a year for a particular street
         - street_df['capacity'] = max(street_df['spots_taken'])
         - perhaps also apply a proprotion in front to compensate for the overpurchasing? Call it a 'gamma' value and perhaps set it to .9? Not totally sure on this part.
         - alternatively, could take the 75th percentile or something instead.  More cumbersome though